# Sheet properties

In [15]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Imports

In [16]:
import sys
sys.path.append('/home/rcendre/classification')
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="1";  
from joblib import dump, load
from numpy import array, logspace
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from toolbox.classification.common import Data, Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.transforms.common import FlattenTransform
from toolbox.transforms.labels import OrderedEncoder
from toolbox.views.common import Views, ViewsTools
from toolbox.models.models import DecisionVotingClassifier, ScoreVotingClassifier
from toolbox.transforms.images import HaralickImageTransform, SpatialImageTransform
import warnings
warnings.filterwarnings('ignore') 

# Parameters

In [17]:
# Advanced parameters
validation = 4
settings = Settings.get_default_dermatology()

# Patch classifier

In [18]:
patch = Dermatology.images(modality='Microscopy', data_type='Patch', use_unknown=False)
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(patch, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Transform groups
group_encoder = LabelEncoder().fit(array(patch['ID_Patient'].tolist()))
Tools.transform(patch, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Extract features 
method = Applications.get_transfer_learning(architecture=architecture_name, pooling='avg', batch_size=1, additional=architecture_additional)
method.transform = method.predict_proba
Tools.transform(patch, {'datum': 'Datum'}, method, 'Transfert')
# Make folds
Folds.build_group_folds(patch, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)
IO.save(patch, f'Instances_PatchTransfert.pickle')

In [19]:
low_predictor = Pipeline([('scale', MinMaxScaler()),('clf', SVC(kernel='linear', class_weight='balanced', decision_function_shape='ovo', probability=True))])
low_params = {'clf__C': logspace(-2, 3, 6).tolist()}

## Evaluate

In [20]:
Tools.evaluate(patch, {'datum': 'Transfert', 'label_encode': 'LabelEncode'}, low_predictor, 'Evaluate', distribution=low_params)

In [21]:
from IPython.display import HTML
from IPython.display import display
display(HTML(ViewsTools.dataframe_renderer([Views.report(patch, {'label_encode': 'LabelEncode', 'eval': 'Evaluate'}, label_encoder)],
                                                                        title=[f'Test - Patch SVC'])))

,precision,recall,f1-score,support
Normal,0.86±0.09,0.72±0.12,0.79±0.10,1880.00±143.60
Benign,0.73±0.10,0.64±0.15,0.69±0.11,2332.00±179.52
Malignant,0.33±0.16,0.53±0.09,0.41±0.13,936.00±62.70
accuracy,0.65±0.11,0.65±0.11,0.65±0.11,0.65±0.11
macro avg,0.64±0.07,0.63±0.09,0.63±0.09,5148.00±0.71
weighted avg,0.71±0.08,0.65±0.11,0.67±0.10,5148.00±0.71


## Fit

In [22]:
low_predictor = Tools.fit(patch, {'datum': 'Transfert', 'label_encode': 'LabelEncode'}, low_predictor, distribution=low_params)

In [23]:
dump(low_predictor, 'Instances_TransfertPredictor')

['Instances_SpatialPredictor']

# Sliding window

In [24]:
inputs_options = []
inputs_options.append((250,0))
inputs_options.append((250,25))
inputs_options.append((250,50))
inputs_options.append((500,0))
inputs_options.append((500,25))
inputs_options.append((500,50))

In [ ]:
low_predictor = load('Instances_TransfertPredictor')
for size, overlap in inputs_options:
    inputs = Dermatology.sliding_images(size=size, overlap=overlap/100, modality='Microscopy')
    Tools.transform(inputs, {'datum': 'Datum'}, HaralickImageTransform(), 'Transfert')
    # Transform groups
    group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
    Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
    # Transform labels
    label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
    Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
    # Make folds
    Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)
    IO.save(inputs, f'Instances_Transfert_{size}_{overlap}.pickle')

# Low level predictions

In [ ]:
for size, overlap in inputs_options:
    inputs = IO.load(f'Instances_Transfert_{size}_{overlap}.pickle')
    
    # Add scaling
    Tools.predict(inputs, {'datum': 'Transfert'}, low_predictor, Tools.PREDICTION, mask=inputs.Type != 'Full')
    Tools.predict_proba(inputs, {'datum': 'Transfert'}, low_predictor, Tools.PROBABILITY, mask=inputs.Type != 'Full')

    Data.build_bags(inputs, inputs.Type == 'Full', 'Reference', inputs.Type == 'Instance', 'Source', Tools.PREDICTION)
    Data.build_bags(inputs, inputs.Type == 'Full', 'Reference', inputs.Type == 'Instance', 'Source', Tools.PROBABILITY)

    # Filter elements
    inputs = inputs[inputs.Type == 'Full']
    
    # Low Level
    IO.save(inputs, f'Instances_LowTransfert_{size}_{overlap}.pickle')

# High level predictions

In [ ]:
for size, overlap in inputs_options:
    inputs = IO.load(f'Instances_LowTransfert_{size}_{overlap}.pickle')
    
    # Decisions
    Tools.evaluate(inputs, {'datum': Tools.PREDICTION, 'label_encode': 'LabelEncode'}, DecisionVotingClassifier(mode='at_least_one', metric=f1_score), 'D_ALO')
    Tools.evaluate(inputs, {'datum': Tools.PREDICTION, 'label_encode': 'LabelEncode'}, DecisionVotingClassifier(mode='dynamic_thresh', metric=f1_score), 'D_DYN')

    # Score
    Tools.evaluate(inputs, {'datum': Tools.PROBABILITY, 'label_encode': 'LabelEncode'}, ScoreVotingClassifier(low='max', high='dynamic'), 'S_MAX')
    
    # SVC
    score_svc = Pipeline([('flat', FlattenTransform()), ('clf', SVC(probability=True))])
    score_params = {'clf__C': logspace(-2, 3, 6).tolist()}
    Tools.evaluate(inputs, {'datum': Tools.PROBABILITY, 'label_encode': 'LabelEncode'}, score_svc, 'High_SVC', distribution=score_params)

    # High Level    
    IO.save(inputs, f'Instances_HighTransfert_{size}_{overlap}.pickle')

## Analysis

In [ ]:
from IPython.display import HTML
from IPython.display import display

label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
models = ['D_ALO', 'D_DYN', 'S_MAX', 'High_SVC']

for size, overlap in inputs_options:
    inputs = IO.load(f'Instances_HighTransfert_{size}_{overlap}.pickle')
    
    for high_name in models:
        name = f'{high_name}_{size}_{overlap}'
        # Details and results
        display(HTML(ViewsTools.dataframe_renderer([Views.report(inputs, {'label_encode': 'LabelEncode', 'eval': high_name}, label_encoder)],
                                                                    title=[f'Test - {name}']))) 